In [1]:
import pandas as pd
complete_data = pd.read_csv('complete_data.csv')
relevant_columns = [
    'Date', 'Branch', 'Location', 'Feeder', 'Category', 'Fault', 'Areas Affected', 'Peak Generation',
    'Daily Energy Generation', 'Lowest Energy Generation', 'Daily Energy Sent', '6:00 Generation',
    'Highest System Frequency', 'Lowest System Frequency', 'Highest Voltage Recorded', 'Lowest Voltage Recorded',
    'temp', 'humidity',
    'precip', 'precipprob', 'precipcover', 'windgust', 'windspeed', 'winddir', 'sealevelpressure',
    'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
    'conditions', 'description', 'icon', 'feelslike'
]
complete_data['Outage'] = complete_data['Category'].apply(lambda x: 0 if pd.isna(x) else 1)
initial_data = complete_data[relevant_columns]

### Fault Type Prediction

In [2]:
X_nlp = initial_data["Fault"]
y_nlp = initial_data["Category"]
# Replace 'Transformer (N/S)' with 'Transformer' in y_nlp
y_nlp.replace({'Transformer (N/S)': 'Transformer'}, inplace=True)
y_nlp.replace({'HVDS': 'High-Voltage (Distribution) System'}, inplace=True)
y_nlp.replace({'TCN': 'Transmission'}, inplace=True)
y_nlp.replace({'Notification': 'Customer Notification'}, inplace=True)
y_nlp.replace({'DT': 'Transformer'}, inplace=True)

/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32787/287174989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_nlp.replace({'Transformer (N/S)': 'Transformer'}, inplace=True)
/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32787/287174989.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_nlp.replace({'HVDS': 'High-Voltage (Distribution) System'}, inplace=True)
/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32787/287174989.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [3]:
y_nlp

0            Feeder
1            Feeder
2               NaN
3       Transformer
4               NaN
           ...     
1047            NaN
1048            NaN
1049         Feeder
1050     Substation
1051     Substation
Name: Category, Length: 1052, dtype: object

In [4]:
# Get indices of NaN values in X
dropped_indices = X_nlp[X_nlp.isna()].index

# Drop NaN values from X and y
X_nlp.dropna(inplace=True)
y_nlp.drop(dropped_indices, inplace=True)

/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32787/810286693.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_nlp.dropna(inplace=True)
/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32787/810286693.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_nlp.drop(dropped_indices, inplace=True)


In [5]:
X_nlp

0                              FAULT ON OWORO IGBOBI 33KV
1                              FAULT ON OWORO IGBOBI 33KV
3       FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...
9                          FAULT ON AGBE ROAD 11KV FEEDER
10                         FAULT ON AGBE ROAD 11KV FEEDER
                              ...                        
1044                   OUTAGE ON 11-OPEBIINJ-T1-SALVATION
1045    REQUEST TO NOTIFY CUSTOMERS ON MATOGUN 11KV FE...
1049                         OUTAGE ON IGANDO 33KV FEEDER
1050    CNN TO ALL CUSTOMERS FEEDING FROM ALIMOSHO 3X1...
1051    CNN TO ALL CUSTOMERS FEEDING FROM AMIKANLE 1X1...
Name: Fault, Length: 558, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [7]:
# Generating this myself for "human" induced power outages which includes vandalism, cyber attacks etc.
human = [
        'Data breach affecting power distribution',
        'Hacking incident targeting grid operations',
        'Stolen devices disrupting energy supply',
        'Vandalism on critical power infrastructure',
        'Cyber intrusion causing network outage',
        'Equipment theft impacting service reliability',
        'Malicious cyber attack on power systems',
        'Sabotage leading to power grid failure',
        'Cyber security breach affecting electricity delivery',
        'Damage to power lines by vandals'
    ]

y_human = ['Human'] * 10 

# Convert human-related data to pandas Series
X_human = pd.Series(human)
# Concatenate existing X_nlp and X_human
X_total = pd.concat([X_nlp, X_human], ignore_index=True)

# Convert human-related data to pandas Series
y_human = pd.Series(y_human)
# Concatenate existing X_nlp and X_human
y_total = pd.concat([y_nlp, y_human], ignore_index=True)

In [8]:
y_total

0           Feeder
1           Feeder
2      Transformer
3           Feeder
4           Feeder
          ...     
563          Human
564          Human
565          Human
566          Human
567          Human
Length: 568, dtype: object

In [76]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 136.3 kB/s eta 0:00:00a 0:00:01


In [9]:
import string
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already done
nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

# Define a function for text preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply text preprocessing to X_nlp
X_processed = X_nlp.apply(preprocess_text)


[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [10]:
X_nlp

0                              FAULT ON OWORO IGBOBI 33KV
1                              FAULT ON OWORO IGBOBI 33KV
3       FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...
9                          FAULT ON AGBE ROAD 11KV FEEDER
10                         FAULT ON AGBE ROAD 11KV FEEDER
                              ...                        
1044                   OUTAGE ON 11-OPEBIINJ-T1-SALVATION
1045    REQUEST TO NOTIFY CUSTOMERS ON MATOGUN 11KV FE...
1049                         OUTAGE ON IGANDO 33KV FEEDER
1050    CNN TO ALL CUSTOMERS FEEDING FROM ALIMOSHO 3X1...
1051    CNN TO ALL CUSTOMERS FEEDING FROM AMIKANLE 1X1...
Name: Fault, Length: 558, dtype: object

In [11]:
X_processed

0                            fault oworo igbobi 33kv
1                            fault oworo igbobi 33kv
3       fault 11ijaiye ojokoroinjt1agbado 2majeobaje
9                        fault agbe road 11kv feeder
10                       fault agbe road 11kv feeder
                            ...                     
1044                    outage 11opebiinjt1salvation
1045    request notify customers matogun 11kv feeder
1049                       outage igando 33kv feeder
1050      cnn customers feeding alimosho 3x15mva iss
1051      cnn customers feeding amikanle 1x15mva iss
Name: Fault, Length: 558, dtype: object

In [13]:
y_pred

array(['Customer Notification', 'Feeder', 'Breaker', 'Transformer',
       'Transformer', 'Feeder', 'Transformer', 'Planned', 'Feeder',
       'Feeder', 'Feeder', 'Transformer', 'Planned',
       'Customer Notification', 'Feeder', 'Transformer',
       'Customer Notification', 'Breaker', 'Feeder', 'Feeder', 'Planned',
       'Transformer', 'Feeder', 'Transformer', 'Planned', 'Feeder',
       'Transformer', 'Transformer', 'Planned', 'Planned', 'Transformer',
       'Transmission', 'Feeder', 'Feeder', 'Transformer', 'Feeder',
       'Feeder', 'Transformer', 'Planned', 'Customer Notification',
       'Transformer', 'High-Voltage (Distribution) System',
       'Customer Notification', 'Customer Notification', 'Transformer',
       'Planned', 'Feeder', 'Feeder', 'Feeder', 'Customer Notification',
       'Transformer', 'Customer Notification', 'Customer Notification',
       'Customer Notification', 'Customer Notification', 'Planned',
       'Feeder', 'Feeder', 'Transformer', 'Customer Notif

In [12]:
X_processed.index

Index([   0,    1,    3,    9,   10,   24,   35,   36,   38,   47,
       ...
       1034, 1035, 1036, 1038, 1043, 1044, 1045, 1049, 1050, 1051],
      dtype='int64', length=558)

In [13]:
X_human

0             Data breach affecting power distribution
1           Hacking incident targeting grid operations
2              Stolen devices disrupting energy supply
3           Vandalism on critical power infrastructure
4               Cyber intrusion causing network outage
5        Equipment theft impacting service reliability
6              Malicious cyber attack on power systems
7               Sabotage leading to power grid failure
8    Cyber security breach affecting electricity de...
9                     Damage to power lines by vandals
dtype: object

In [14]:
X_total = pd.concat([X_nlp, X_human], ignore_index=True)

In [15]:
y_nlp

0                      Feeder
1                      Feeder
3                 Transformer
9                      Feeder
10                     Feeder
                ...          
1044                   Feeder
1045    Customer Notification
1049                   Feeder
1050               Substation
1051               Substation
Name: Category, Length: 558, dtype: object

In [16]:
from sklearn.metrics import accuracy_score
# Step 2: Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_processed = X_total.apply(preprocess_text)
X_vectorized = tfidf_vectorizer.fit_transform(X_processed)

# Step 3: Model Selection
model = LogisticRegression(penalty='l2', C=10.0, solver='liblinear', max_iter=100,  random_state=42,
                             class_weight='balanced', dual=False)

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_total, test_size=0.2, random_state=6)

# Step 5: Model Training
model.fit(X_train, y_train)

# Step 6: Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(classification_report(y_test, y_pred))


Accuracy: 85.09%
                                    precision    recall  f1-score   support

                           Breaker       1.00      0.75      0.86         4
             Customer Notification       0.94      0.89      0.92        19
                            Feeder       0.83      0.80      0.81        30
High-Voltage (Distribution) System       1.00      1.00      1.00         2
                             Human       0.83      0.83      0.83         6
                           Planned       0.57      0.89      0.70         9
                        Substation       1.00      1.00      1.00         1
                       Transformer       0.89      0.89      0.89        38
                      Transmission       1.00      0.60      0.75         5

                          accuracy                           0.85       114
                         macro avg       0.90      0.85      0.86       114
                      weighted avg       0.87      0.85      0.85    

In [17]:
# Identify misclassified instances
misclassified_indices = (y_test != y_pred)
misclassified_samples = X_test[misclassified_indices]
misclassified_true_labels = y_test[misclassified_indices]
misclassified_pred_labels = y_pred[misclassified_indices]

# To get the original texts/features
# Assuming X_total and X_vectorized are already defined
# Map the indices to the original dataset
original_indices = y_test.index[misclassified_indices]
original_texts = X_total.iloc[original_indices]

# Display misclassified instances
misclassified_df = pd.DataFrame({
    'Original Text': original_texts,
    'True Label': misclassified_true_labels,
    'Predicted Label': misclassified_pred_labels
})

print("Misclassified Instances with Original Texts:")
print(misclassified_df.head(10))  # Display the first 10 misclassified samples

Misclassified Instances with Original Texts:
                                         Original Text             True Label  \
56      PLANNED OUTAGE REQUEST ON T1 300MVA OKE-ARO TS            Transformer   
15       PLANNED OUTAGE ON MARYLAND-AJEGUNLE 33KV LINE                 Feeder   
106            NO SUPPLY ON 11-OGUDUINJ-T1-CAC-LSDPC 3            Transformer   
80       PLANNED OUTAGE REQUEST ON ILUPEJU 33KV FEEDER                 Feeder   
277     PLANNED OUTAGE REQUEST ON AMIKANLE 33KV FEEDER                 Feeder   
457  HT POLE TO ERECT: CNN TO OUR CUSTOMER ON KETU,...  Customer Notification   
334                 PLANNED OUTAGE ON EGBE 33KV FEEDER                 Feeder   
247              PLANNED OUTAGE ON T6 60MVA OKE ARO TS                Planned   
230  tRIPPING ON T3-60MVA OGBA TCN AFFECTING ADENIY...           Transmission   
496         SMS TO OKO-OBA 11 KV CUSTOMER ON AGEGE INJ  Customer Notification   

    Predicted Label  
56          Planned  
15          Planned

In [18]:
misclassified_df

,Original Text,True Label,Predicted Label
56,PLANNED OUTAGE REQUEST ON T1 300MVA OKE-ARO TS,Transformer,Planned
15,PLANNED OUTAGE ON MARYLAND-AJEGUNLE 33KV LINE,Feeder,Planned
106,NO SUPPLY ON 11-OGUDUINJ-T1-CAC-LSDPC 3,Transformer,Feeder
80,PLANNED OUTAGE REQUEST ON ILUPEJU 33KV FEEDER,Feeder,Planned
277,PLANNED OUTAGE REQUEST ON AMIKANLE 33KV FEEDER,Feeder,Planned
457,"HT POLE TO ERECT: CNN TO OUR CUSTOMER ON KETU,...",Customer Notification,Human
334,PLANNED OUTAGE ON EGBE 33KV FEEDER,Feeder,Planned
247,PLANNED OUTAGE ON T6 60MVA OKE ARO TS,Planned,Transformer
230,tRIPPING ON T3-60MVA OGBA TCN AFFECTING ADENIY...,Transmission,Transformer
496,SMS TO OKO-OBA 11 KV CUSTOMER ON AGEGE INJ,Customer Notification,Feeder


In [19]:
from sklearn.metrics import (
    accuracy_score, confusion_matrix, cohen_kappa_score,
    matthews_corrcoef, hamming_loss, log_loss, roc_auc_score, 
    top_k_accuracy_score, balanced_accuracy_score
)
from sklearn.preprocessing import label_binarize

# Model predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Metrics calculations
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
hamming = hamming_loss(y_test, y_pred)
logloss = log_loss(y_test, y_pred_proba)
y_test_binarized = label_binarize(y_test, classes=model.classes_)
auc_roc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovr')
top_k_acc = top_k_accuracy_score(y_test, y_pred_proba, k=3)
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# Print results
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:\n', cm)
print(f'Cohen\'s Kappa: {kappa:.2f}')
print(f'Matthews Correlation Coefficient: {mcc:.2f}')
print(f'Hamming Loss: {hamming:.2f}')
print(f'Log Loss: {logloss:.2f}')
print(f'AUC-ROC: {auc_roc:.2f}')
print(f'Top-3 Accuracy: {top_k_acc:.2f}')
print(f'Balanced Accuracy: {balanced_acc:.2f}')


Accuracy: 85.09%
Confusion Matrix:
 [[ 3  0  0  0  0  0  0  1  0]
 [ 0 17  1  0  1  0  0  0  0]
 [ 0  1 24  0  0  5  0  0  0]
 [ 0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  1  0]
 [ 0  0  0  0  0  8  0  1  0]
 [ 0  0  0  0  0  0  1  0  0]
 [ 0  0  3  0  0  1  0 34  0]
 [ 0  0  1  0  0  0  0  1  3]]
Cohen's Kappa: 0.81
Matthews Correlation Coefficient: 0.81
Hamming Loss: 0.15
Log Loss: 0.56
AUC-ROC: 0.99
Top-3 Accuracy: 0.98
Balanced Accuracy: 0.85


In [17]:
import joblib
# Save the trained model
joblib.dump(model, 'Fault_Classification_Model.pkl')

# Save the TfidfVectorizer. This is used to load with the model in the code.
joblib.dump(tfidf_vectorizer, 'Fault_Classification_tfidf_vectorizer.pkl')

['Fault_Classification_tfidf_vectorizer.pkl']